# Chapter 47: Building a SaaS Dashboard

Software-as-a-Service applications introduce architectural complexity beyond standard web apps: multi-tenancy requires strict data isolation, subscriptions demand reliable billing integration, and real-time collaboration expects instantaneous data synchronization. This chapter demonstrates how to build a scalable SaaS platform using Next.js with organization-based tenancy, Stripe subscriptions for monetization, Server-Sent Events for live analytics, and sophisticated access control systems.

By the end of this chapter, you'll architect multi-tenant database schemas with row-level security, implement organization-aware authentication flows, manage Stripe subscription lifecycles with webhook handling, build real-time analytics dashboards with SSE streaming, enforce feature limitations based on subscription tiers, and create team invitation systems with role-based permissions.

## 47.1 Multi-tenant Architecture

Designing data isolation strategies that scale from startup to enterprise.

### Database Schema for Multi-tenancy

Shared database with tenant isolation via foreign keys:

```typescript
// prisma/schema.prisma
model Account {
  id                String  @id @default(cuid())
  userId            String
  type              String
  provider          String
  providerAccountId String
  user              User    @relation(fields: [userId], references: [id], onDelete: Cascade)
  
  @@unique([provider, providerAccountId])
}

model User {
  id            String    @id @default(cuid())
  email         String    @unique
  name          String?
  image         String?
  emailVerified DateTime?
  accounts      Account[]
  sessions      Session[]
  
  // Multi-tenancy: User can belong to multiple organizations
  memberships   OrganizationMember[]
  ownedOrgs     Organization[]       @relation("OrganizationOwner")
  
  createdAt     DateTime             @default(now())
}

model Session {
  id           String   @id @default(cuid())
  sessionToken String   @unique
  userId       String
  expires      DateTime
  user         User     @relation(fields: [userId], references: [id], onDelete: Cascade)
}

model Organization {
  id          String   @id @default(cuid())
  name        String
  slug        String   @unique
  image       String?
  
  // Subscription & Billing
  stripeCustomerId       String? @unique
  stripeSubscriptionId   String? @unique
  stripePriceId          String?
  subscriptionStatus     SubscriptionStatus @default(INCOMPLETE)
  currentPeriodStart     DateTime?
  currentPeriodEnd       DateTime?
  plan                   PlanType @default(FREE)
  
  // Relations
  ownerId     String
  owner       User     @relation("OrganizationOwner", fields: [ownerId], references: [id])
  members     OrganizationMember[]
  projects    Project[]
  apiKeys     ApiKey[]
  auditLogs   AuditLog[]
  
  // Usage tracking
  usage       UsageRecord[]
  
  createdAt   DateTime @default(now())
  updatedAt   DateTime @updatedAt

  @@index([slug])
  @@index([stripeCustomerId])
  @@index([subscriptionStatus])
}

model OrganizationMember {
  id             String @id @default(cuid())
  organizationId String
  userId         String
  role           OrgRole @default(MEMBER)
  
  organization   Organization @relation(fields: [organizationId], references: [id], onDelete: Cascade)
  user           User         @relation(fields: [userId], references: [id], onDelete: Cascade)
  
  joinedAt       DateTime @default(now())

  @@unique([organizationId, userId])
  @@index([userId])
}

model Project {
  id             String @id @default(cuid())
  name           String
  description    String?
  organizationId String
  
  organization   Organization @relation(fields: [organizationId], references: [id], onDelete: Cascade)
  environments   Environment[]
  deployments    Deployment[]
  
  createdAt      DateTime @default(now())

  @@index([organizationId])
}

// Audit trail for compliance
model AuditLog {
  id             String   @id @default(cuid())
  organizationId String
  userId         String?
  action         String   // e.g., "project.created", "member.invited"
  entityType     String   // "project", "member", "api_key"
  entityId       String?
  metadata       Json?
  ipAddress      String?
  userAgent      String?
  
  organization   Organization @relation(fields: [organizationId], references: [id], onDelete: Cascade)
  createdAt      DateTime     @default(now())

  @@index([organizationId, createdAt])
  @@index([action])
}

enum SubscriptionStatus {
  INCOMPLETE
  ACTIVE
  PAST_DUE
  CANCELED
  UNPAID
}

enum PlanType {
  FREE
  STARTER
  PRO
  ENTERPRISE
}

enum OrgRole {
  OWNER
  ADMIN
  MEMBER
  VIEWER
}
```

### Tenant Resolution Strategy

Middleware-based organization context injection:

```typescript
// middleware.ts
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';
import { getToken } from 'next-auth/jwt';
import { prisma } from '@/shared/lib/db';

export async function middleware(request: NextRequest) {
  const pathname = request.nextUrl.pathname;
  
  // Skip auth routes and static files
  if (
    pathname.startsWith('/api/auth') ||
    pathname.startsWith('/_next') ||
    pathname.startsWith('/static')
  ) {
    return NextResponse.next();
  }

  // Extract organization slug from subdomain or path
  const hostname = request.headers.get('host') || '';
  const subdomain = hostname.split('.')[0];
  
  // Check for org slug in subdomain (acme.yoursaas.com)
  let orgSlug = null;
  if (!hostname.includes('localhost') && !hostname.includes('vercel.app')) {
    orgSlug = subdomain;
  } else {
    // Fall back to path-based (/app/[orgSlug]/...)
    const match = pathname.match(/\/app\/([^\/]+)/);
    if (match) orgSlug = match[1];
  }

  if (!orgSlug) return NextResponse.next();

  // Validate organization exists and is active
  const org = await prisma.organization.findUnique({
    where: { slug: orgSlug },
    select: { id: true, subscriptionStatus: true, plan: true },
  });

  if (!org) {
    return NextResponse.redirect(new URL('/404', request.url));
  }

  // Check subscription status for paid features
  if (pathname.includes('/settings/billing') && org.subscriptionStatus === 'PAST_DUE') {
    // Allow access to billing even when past due
    return NextResponse.next();
  }

  if (org.subscriptionStatus === 'CANCELED' || org.subscriptionStatus === 'UNPAID') {
    return NextResponse.redirect(new URL(`/${orgSlug}/billing/reactivate`, request.url));
  }

  // Add organization context to headers for Server Components
  const requestHeaders = new Headers(request.headers);
  requestHeaders.set('x-organization-id', org.id);
  requestHeaders.set('x-organization-plan', org.plan);

  return NextResponse.next({
    request: { headers: requestHeaders },
  });
}

export const config = {
  matcher: ['/app/:path*', '/api/app/:path*'],
};
```

## 47.2 Authentication & Authorization

Organization-aware access control with hierarchical permissions.

### Organization Context Provider

```typescript
// app/app/[orgSlug]/layout.tsx
import { notFound, redirect } from 'next/navigation';
import { getServerSession } from 'next-auth';
import { authOptions } from '@/app/api/auth/[...nextauth]/route';
import { prisma } from '@/shared/lib/db';
import { OrganizationProvider } from '@/features/organization/organization-context';
import { AppShell } from '@/features/layout/app-shell';

export default async function OrgLayout({
  children,
  params,
}: {
  children: React.ReactNode;
  params: { orgSlug: string };
}) {
  const session = await getServerSession(authOptions);
  
  if (!session?.user) {
    redirect(`/auth/signin?callbackUrl=/app/${params.orgSlug}`);
  }

  // Fetch organization with membership
  const organization = await prisma.organization.findUnique({
    where: { slug: params.orgSlug },
    include: {
      members: {
        where: { userId: session.user.id },
        select: { role: true },
      },
    },
  });

  if (!organization) notFound();
  
  // Check membership
  const membership = organization.members[0];
  if (!membership) {
    redirect('/app'); // User doesn't belong to this org
  }

  return (
    <OrganizationProvider 
      organization={organization} 
      userRole={membership.role}
    >
      <AppShell orgSlug={params.orgSlug}>
        {children}
      </AppShell>
    </OrganizationProvider>
  );
}

// features/organization/organization-context.tsx
'use client';

import { createContext, useContext, ReactNode } from 'react';

interface OrganizationContextType {
  id: string;
  name: string;
  slug: string;
  plan: string;
  userRole: string;
  isOwner: boolean;
  isAdmin: boolean;
}

const OrganizationContext = createContext<OrganizationContextType | null>(null);

export function OrganizationProvider({ 
  children, 
  organization,
  userRole,
}: { 
  children: ReactNode;
  organization: any;
  userRole: string;
}) {
  return (
    <OrganizationContext.Provider
      value={{
        ...organization,
        userRole,
        isOwner: userRole === 'OWNER',
        isAdmin: userRole === 'ADMIN' || userRole === 'OWNER',
      }}
    >
      {children}
    </OrganizationContext.Provider>
  );
}

export const useOrganization = () => {
  const context = useContext(OrganizationContext);
  if (!context) throw new Error('useOrganization must be used within OrganizationProvider');
  return context;
};
```

### Permission-based Access Control

```typescript
// lib/permissions.ts
import { OrgRole } from '@prisma/client';

type Permission = 
  | 'project:create'
  | 'project:delete'
  | 'project:update'
  | 'member:invite'
  | 'member:remove'
  | 'member:update_role'
  | 'billing:manage'
  | 'settings:manage'
  | 'api_keys:manage';

const rolePermissions: Record<OrgRole, Permission[]> = {
  OWNER: [
    'project:create', 'project:delete', 'project:update',
    'member:invite', 'member:remove', 'member:update_role',
    'billing:manage', 'settings:manage', 'api_keys:manage'
  ],
  ADMIN: [
    'project:create', 'project:update',
    'member:invite', 'member:remove',
    'settings:manage', 'api_keys:manage'
  ],
  MEMBER: [
    'project:create', 'project:update'
  ],
  VIEWER: []
};

export function hasPermission(role: OrgRole, permission: Permission): boolean {
  return rolePermissions[role].includes(permission);
}

// Server-side permission check
export async function requirePermission(
  orgId: string, 
  userId: string, 
  permission: Permission
) {
  const membership = await prisma.organizationMember.findUnique({
    where: {
      organizationId_userId: {
        organizationId: orgId,
        userId,
      },
    },
  });

  if (!membership || !hasPermission(membership.role, permission)) {
    throw new Error('Insufficient permissions');
  }

  return membership;
}

// Usage in Server Actions
// features/projects/actions.ts
'use server';

export async function createProject(formData: FormData) {
  const session = await getServerSession(authOptions);
  const orgId = headers().get('x-organization-id')!;
  
  await requirePermission(orgId, session.user.id, 'project:create');
  
  // Proceed with creation...
}
```

## 47.3 Subscription Management

Stripe integration for recurring billing and plan enforcement.

### Stripe Subscription Lifecycle

```typescript
// app/api/stripe/checkout/route.ts
import { stripe } from '@/shared/lib/stripe';
import { getServerSession } from 'next-auth';
import { authOptions } from '@/app/api/auth/[...nextauth]/route';
import { prisma } from '@/shared/lib/db';

export async function POST(request: Request) {
  try {
    const session = await getServerSession(authOptions);
    if (!session?.user) {
      return new Response('Unauthorized', { status: 401 });
    }

    const { priceId, orgId } = await request.json();

    // Get or create Stripe customer
    const org = await prisma.organization.findUnique({
      where: { id: orgId },
      include: { members: { where: { userId: session.user.id } } }
    });

    if (!org?.members[0]?.role === 'OWNER') {
      return new Response('Forbidden', { status: 403 });
    }

    let customerId = org.stripeCustomerId;
    if (!customerId) {
      const customer = await stripe.customers.create({
        email: session.user.email!,
        name: org.name,
        metadata: { organizationId: org.id },
      });
      customerId = customer.id;
      
      await prisma.organization.update({
        where: { id: orgId },
        data: { stripeCustomerId: customerId },
      });
    }

    // Create checkout session
    const checkoutSession = await stripe.checkout.sessions.create({
      customer: customerId,
      mode: 'subscription',
      line_items: [{ price: priceId, quantity: 1 }],
      success_url: `${process.env.NEXT_PUBLIC_URL}/app/${org.slug}/settings/billing?success=true`,
      cancel_url: `${process.env.NEXT_PUBLIC_URL}/app/${org.slug}/settings/billing?canceled=true`,
      subscription_data: {
        metadata: { organizationId: orgId },
      },
    });

    return Response.json({ url: checkoutSession.url });
  } catch (error) {
    console.error('Checkout error:', error);
    return new Response('Internal error', { status: 500 });
  }
}
```

### Webhook Handling for Subscription Updates

```typescript
// app/api/stripe/webhook/route.ts
import { NextResponse } from 'next/server';
import { headers } from 'next/headers';
import { stripe } from '@/shared/lib/stripe';
import { prisma } from '@/shared/lib/db';
import { revalidatePath } from 'next/cache';

const webhookSecret = process.env.STRIPE_WEBHOOK_SECRET!;

export async function POST(request: Request) {
  const payload = await request.text();
  const signature = headers().get('stripe-signature')!;

  let event;
  try {
    event = stripe.webhooks.constructEvent(payload, signature, webhookSecret);
  } catch (err: any) {
    return NextResponse.json({ error: err.message }, { status: 400 });
  }

  switch (event.type) {
    case 'checkout.session.completed': {
      const session = event.data.object;
      const orgId = session.metadata?.organizationId;
      
      if (orgId) {
        await prisma.organization.update({
          where: { id: orgId },
          data: {
            subscriptionStatus: 'ACTIVE',
            stripeSubscriptionId: session.subscription as string,
          },
        });
        
        revalidatePath(`/app/[orgSlug]/settings/billing`);
      }
      break;
    }

    case 'invoice.payment_failed': {
      const invoice = event.data.object;
      const org = await prisma.organization.findFirst({
        where: { stripeCustomerId: invoice.customer as string },
      });
      
      if (org) {
        await prisma.organization.update({
          where: { id: org.id },
          data: { subscriptionStatus: 'PAST_DUE' },
        });
        
        // Notify organization owners
        await notifyPaymentFailure(org.id);
      }
      break;
    }

    case 'customer.subscription.deleted': {
      const subscription = event.data.object;
      const org = await prisma.organization.findFirst({
        where: { stripeSubscriptionId: subscription.id as string },
      });
      
      if (org) {
        await prisma.organization.update({
          where: { id: org.id },
          data: {
            subscriptionStatus: 'CANCELED',
            plan: 'FREE',
            stripeSubscriptionId: null,
            stripePriceId: null,
          },
        });
      }
      break;
    }

    case 'customer.subscription.updated': {
      const subscription = event.data.object;
      const org = await prisma.organization.findFirst({
        where: { stripeSubscriptionId: subscription.id as string },
      });
      
      if (org) {
        const priceId = subscription.items.data[0].price.id;
        const plan = getPlanFromPriceId(priceId);
        
        await prisma.organization.update({
          where: { id: org.id },
          data: {
            plan,
            subscriptionStatus: subscription.status.toUpperCase() as any,
            currentPeriodStart: new Date(subscription.current_period_start * 1000),
            currentPeriodEnd: new Date(subscription.current_period_end * 1000),
          },
        });
      }
      break;
    }
  }

  return NextResponse.json({ received: true });
}

function getPlanFromPriceId(priceId: string): string {
  const map: Record<string, string> = {
    [process.env.STRIPE_STARTER_PRICE_ID!]: 'STARTER',
    [process.env.STRIPE_PRO_PRICE_ID!]: 'PRO',
    [process.env.STRIPE_ENTERPRISE_PRICE_ID!]: 'ENTERPRISE',
  };
  return map[priceId] || 'FREE';
}
```

## 47.4 Real-time Analytics

Live dashboard updates using Server-Sent Events.

### Analytics Aggregation

```typescript
// lib/analytics/aggregation.ts
import { prisma } from '@/shared/lib/db';

export async function getRealtimeMetrics(orgId: string) {
  const now = new Date();
  const oneHourAgo = new Date(now.getTime() - 60 * 60 * 1000);
  
  const [currentUsers, eventsLastHour, topProjects] = await Promise.all([
    // Active users in last 5 minutes
    prisma.auditLog.groupBy({
      by: ['userId'],
      where: {
        organizationId: orgId,
        createdAt: { gte: new Date(now.getTime() - 5 * 60 * 1000) },
      },
      _count: true,
    }).then(results => results.length),

    // Event volume
    prisma.auditLog.groupBy({
      by: ['action'],
      where: {
        organizationId: orgId,
        createdAt: { gte: oneHourAgo },
      },
      _count: { action: true },
    }),

    // Top projects by activity
    prisma.project.findMany({
      where: { organizationId: orgId },
      include: {
        _count: {
          select: { deployments: { where: { createdAt: { gte: oneHourAgo } } } },
        },
      },
      orderBy: { deployments: { _count: 'desc' } },
      take: 5,
    }),
  ]);

  return {
    currentUsers,
    eventsLastHour: eventsLastHour.reduce((acc, curr) => {
      acc[curr.action] = curr._count.action;
      return acc;
    }, {} as Record<string, number>),
    topProjects: topProjects.map(p => ({ name: p.name, deployments: p._count.deployments })),
    timestamp: now.toISOString(),
  };
}
```

### SSE Stream Implementation

```typescript
// app/api/app/[orgSlug]/stream/route.ts
import { getServerSession } from 'next-auth';
import { authOptions } from '@/app/api/auth/[...nextauth]/route';
import { prisma } from '@/shared/lib/db';
import { getRealtimeMetrics } from '@/lib/analytics/aggregation';

export const dynamic = 'force-dynamic';

export async function GET(
  request: Request,
  { params }: { params: { orgSlug: string } }
) {
  const session = await getServerSession(authOptions);
  if (!session?.user) {
    return new Response('Unauthorized', { status: 401 });
  }

  const org = await prisma.organization.findUnique({
    where: { slug: params.orgSlug },
    include: { members: { where: { userId: session.user.id } } },
  });

  if (!org?.members.length) {
    return new Response('Forbidden', { status: 403 });
  }

  const encoder = new TextEncoder();
  let interval: NodeJS.Timeout;

  const stream = new ReadableStream({
    start(controller) {
      // Send initial data
      const sendData = async () => {
        try {
          const metrics = await getRealtimeMetrics(org.id);
          const data = `data: ${JSON.stringify(metrics)}\n\n`;
          controller.enqueue(encoder.encode(data));
        } catch (e) {
          console.error('Metrics error:', e);
        }
      };

      sendData();
      
      // Update every 10 seconds
      interval = setInterval(sendData, 10000);

      // Cleanup on disconnect
      request.signal.addEventListener('abort', () => {
        clearInterval(interval);
        controller.close();
      });
    },
    cancel() {
      clearInterval(interval);
    },
  });

  return new Response(stream, {
    headers: {
      'Content-Type': 'text/event-stream',
      'Cache-Control': 'no-cache',
      'Connection': 'keep-alive',
    },
  });
}
```

### Live Dashboard Component

```typescript
// features/analytics/live-dashboard.tsx
'use client';

import { useEffect, useState } from 'react';
import { useOrganization } from '@/features/organization/organization-context';
import { LineChart, BarChart } from '@/shared/ui/charts';

interface Metrics {
  currentUsers: number;
  eventsLastHour: Record<string, number>;
  topProjects: Array<{ name: string; deployments: number }>;
  timestamp: string;
}

export function LiveDashboard() {
  const { slug } = useOrganization();
  const [metrics, setMetrics] = useState<Metrics | null>(null);
  const [connectionStatus, setConnectionStatus] = useState<'connected' | 'disconnected'>('disconnected');

  useEffect(() => {
    const eventSource = new EventSource(`/api/app/${slug}/stream`);

    eventSource.onopen = () => setConnectionStatus('connected');
    eventSource.onerror = () => setConnectionStatus('disconnected');

    eventSource.onmessage = (event) => {
      try {
        const data = JSON.parse(event.data);
        setMetrics(data);
      } catch (e) {
        console.error('Failed to parse metrics:', e);
      }
    };

    return () => eventSource.close();
  }, [slug]);

  if (!metrics) return <DashboardSkeleton />;

  return (
    <div className="space-y-6">
      <div className="flex justify-between items-center">
        <h2 className="text-2xl font-bold">Live Activity</h2>
        <div className="flex items-center gap-2">
          <div className={`w-2 h-2 rounded-full ${connectionStatus === 'connected' ? 'bg-green-500 animate-pulse' : 'bg-red-500'}`} />
          <span className="text-sm text-gray-600">
            {connectionStatus === 'connected' ? 'Live' : 'Reconnecting...'}
          </span>
        </div>
      </div>

      <div className="grid grid-cols-1 md:grid-cols-3 gap-4">
        <MetricCard
          title="Active Users"
          value={metrics.currentUsers}
          subtitle="Currently online"
        />
        <MetricCard
          title="Events (1h)"
          value={Object.values(metrics.eventsLastHour).reduce((a, b) => a + b, 0)}
          subtitle="Total actions"
        />
        <MetricCard
          title="Top Project"
          value={metrics.topProjects[0]?.name || 'N/A'}
          subtitle={`${metrics.topProjects[0]?.deployments || 0} deployments`}
        />
      </div>

      <div className="grid grid-cols-1 lg:grid-cols-2 gap-6">
        <div className="bg-white p-6 rounded-lg border">
          <h3 className="font-semibold mb-4">Event Distribution</h3>
          <BarChart data={metrics.eventsLastHour} />
        </div>
        
        <div className="bg-white p-6 rounded-lg border">
          <h3 className="font-semibold mb-4">Project Activity</h3>
          <LineChart 
            data={metrics.topProjects.map(p => ({ name: p.name, value: p.deployments }))} 
          />
        </div>
      </div>
    </div>
  );
}
```

## 47.5 Feature Gating

Enforcing plan limitations and usage quotas.

### Plan Configuration

```typescript
// config/plans.ts
export const plans = {
  FREE: {
    name: 'Free',
    price: 0,
    limits: {
      projects: 2,
      members: 1,
      apiCalls: 1000, // per month
      storage: 100 * 1024 * 1024, // 100MB
      environments: 2,
      analyticsRetention: 7, // days
    },
    features: {
      customDomain: false,
      sso: false,
      auditLogs: false,
      prioritySupport: false,
    },
  },
  STARTER: {
    name: 'Starter',
    price: 29,
    limits: {
      projects: 10,
      members: 5,
      apiCalls: 10000,
      storage: 1024 * 1024 * 1024, // 1GB
      environments: 5,
      analyticsRetention: 30,
    },
    features: {
      customDomain: true,
      sso: false,
      auditLogs: true,
      prioritySupport: false,
    },
  },
  PRO: {
    name: 'Professional',
    price: 99,
    limits: {
      projects: 50,
      members: 20,
      apiCalls: 100000,
      storage: 10 * 1024 * 1024 * 1024, // 10GB
      environments: 10,
      analyticsRetention: 90,
    },
    features: {
      customDomain: true,
      sso: true,
      auditLogs: true,
      prioritySupport: true,
    },
  },
  ENTERPRISE: {
    name: 'Enterprise',
    price: 'custom',
    limits: {
      projects: Infinity,
      members: Infinity,
      apiCalls: Infinity,
      storage: Infinity,
      environments: Infinity,
      analyticsRetention: 365,
    },
    features: {
      customDomain: true,
      sso: true,
      auditLogs: true,
      prioritySupport: true,
      sla: true,
    },
  },
};

export type PlanType = keyof typeof plans;
```

### Usage Tracking & Enforcement

```typescript
// lib/usage/enforcement.ts
import { prisma } from '@/shared/lib/db';
import { plans, PlanType } from '@/config/plans';

export async function checkLimit(
  orgId: string,
  resource: keyof typeof plans.FREE.limits
) {
  const org = await prisma.organization.findUnique({
    where: { id: orgId },
    include: {
      _count: { select: { projects: true, members: true } },
      usage: { where: { period: new Date().toISOString().slice(0, 7) } }, // Current month
    },
  });

  if (!org) throw new Error('Organization not found');

  const plan = plans[org.plan as PlanType];
  const limit = plan.limits[resource];

  if (limit === Infinity) return { allowed: true, current: 0, limit };

  let current = 0;
  
  switch (resource) {
    case 'projects':
      current = org._count.projects;
      break;
    case 'members':
      current = org._count.members;
      break;
    case 'apiCalls':
      current = org.usage[0]?.apiCalls || 0;
      break;
    case 'storage':
      current = org.usage[0]?.storageBytes || 0;
      break;
  }

  return {
    allowed: current < limit,
    current,
    limit,
    remaining: limit - current,
  };
}

export async function incrementUsage(
  orgId: string,
  metric: 'apiCalls' | 'storageBytes',
  amount: number = 1
) {
  const period = new Date().toISOString().slice(0, 7); // YYYY-MM
  
  await prisma.usageRecord.upsert({
    where: {
      organizationId_period: {
        organizationId: orgId,
        period,
      },
    },
    update: {
      [metric]: { increment: amount },
    },
    create: {
      organizationId: orgId,
      period,
      [metric]: amount,
    },
  });
}

// Middleware usage check example
export async function apiUsageMiddleware(
  orgId: string,
  cost: number = 1
) {
  const check = await checkLimit(orgId, 'apiCalls');
  
  if (!check.allowed) {
    throw new Error('API quota exceeded. Please upgrade your plan.');
  }
  
  await incrementUsage(orgId, 'apiCalls', cost);
}
```

## 47.6 Team Management

Invitation flows and role assignments.

### Invitation System

```typescript
// features/team/actions.ts
'use server';

import { randomBytes } from 'crypto';
import { prisma } from '@/shared/lib/db';
import { sendEmail } from '@/lib/email';
import { requirePermission } from '@/lib/permissions';
import { getServerSession } from 'next-auth';
import { revalidatePath } from 'next/cache';

export async function inviteMember(formData: FormData) {
  const session = await getServerSession(authOptions);
  const orgId = formData.get('orgId') as string;
  const email = formData.get('email') as string;
  const role = formData.get('role') as string;

  await requirePermission(orgId, session!.user.id, 'member:invite');

  // Check plan limits
  const limitCheck = await checkLimit(orgId, 'members');
  if (!limitCheck.allowed) {
    throw new Error(`Member limit reached (${limitCheck.limit}). Upgrade to add more.`);
  }

  // Check if already member
  const existing = await prisma.user.findUnique({
    where: { email },
    include: { memberships: { where: { organizationId: orgId } } },
  });

  if (existing?.memberships.length) {
    throw new Error('User is already a member of this organization');
  }

  // Create invitation token
  const token = randomBytes(32).toString('hex');
  const expires = new Date(Date.now() + 7 * 24 * 60 * 60 * 1000); // 7 days

  await prisma.invitation.create({
    data: {
      email,
      role: role as any,
      token,
      expiresAt: expires,
      organizationId: orgId,
      invitedById: session!.user.id,
    },
  });

  // Send email
  const org = await prisma.organization.findUnique({ where: { id: orgId } });
  
  await sendEmail({
    to: email,
    subject: `You've been invited to join ${org!.name}`,
    html: `
      <p>You've been invited to join ${org!.name} as a ${role}.</p>
      <a href="${process.env.NEXT_PUBLIC_URL}/invitations/${token}">
        Accept Invitation
      </a>
      <p>This link expires in 7 days.</p>
    `,
  });

  revalidatePath(`/app/${org!.slug}/settings/team`);
}

export async function acceptInvitation(token: string) {
  const session = await getServerSession(authOptions);
  if (!session?.user) {
    redirect('/auth/signin');
  }

  const invitation = await prisma.invitation.findUnique({
    where: { token },
    include: { organization: true },
  });

  if (!invitation || invitation.expiresAt < new Date()) {
    throw new Error('Invalid or expired invitation');
  }

  if (invitation.email !== session.user.email) {
    throw new Error('This invitation was sent to a different email address');
  }

  await prisma.$transaction([
    // Add member
    prisma.organizationMember.create({
      data: {
        organizationId: invitation.organizationId,
        userId: session.user.id,
        role: invitation.role,
      },
    }),
    // Delete invitation
    prisma.invitation.delete({ where: { id: invitation.id } }),
    // Log action
    prisma.auditLog.create({
      data: {
        organizationId: invitation.organizationId,
        userId: session.user.id,
        action: 'member.accepted_invite',
        entityType: 'member',
        metadata: { role: invitation.role },
      },
    }),
  ]);

  redirect(`/app/${invitation.organization.slug}`);
}
```

## Key Takeaways from Chapter 47

1. **Multi-tenant Schema Design**: Implement shared-database tenancy using foreign key relationships where every entity (projects, audit logs, usage records) includes an `organizationId`. Create a unique index on `Organization.slug` for subdomain-based routing and store Stripe customer IDs directly on the organization for billing relationship management.

2. **Tenant Resolution**: Use Next.js middleware to extract organization context from subdomains (`acme.yoursaas.com`) or path prefixes (`/app/acme`), validate the user has membership, and inject `x-organization-id` headers for downstream Server Components. Handle subscription status checks in middleware to redirect unpaid accounts to billing reactivation flows.

3. **Hierarchical Permissions**: Define granular permissions (`project:create`, `member:invite`) and map them to roles (OWNER, ADMIN, MEMBER, VIEWER) using a role-based matrix. Implement `requirePermission` Server Action guards that throw immediately if the current user's membership doesn't include the required capability.

4. **Subscription Lifecycle**: Store `stripeSubscriptionId` and `subscriptionStatus` on the Organization model. Handle `checkout.session.completed` webhooks to activate subscriptions, `invoice.payment_failed` to mark accounts past_due, and `customer.subscription.deleted` to downgrade to free plans. Use Stripe metadata to link subscriptions back to organization IDs.

5. **Real-time Analytics**: Implement Server-Sent Events (SSE) endpoints that stream aggregated metrics (active users, event counts, top projects) every 10 seconds. Use Prisma aggregation queries with time-windowed filters (`createdAt: { gte: oneHourAgo }`) and connection pooling for efficient polling without database overload.

6. **Usage Enforcement**: Create a plan configuration object defining limits (projects, members, API calls) and features (SSO, custom domains) per tier. Implement `checkLimit` functions that compare current usage against plan limits before allowing resource creation, and `incrementUsage` calls in API middleware to track consumption against monthly quotas.

7. **Team Collaboration**: Build invitation systems using cryptographically random tokens with 7-day expiration. Send transactional emails with accept links, verify email matching upon acceptance, and wrap member creation + invitation deletion in database transactions to prevent race conditions. Log all membership changes to the audit trail for compliance.

## Coming Up Next

**Chapter 48: Building a Social Media Platform**

Chapter 48 transitions to user-generated content at scale, architecting feed algorithms, real-time notifications, content moderation systems, and follower relationships. You'll implement database patterns for graph-like social connections, build infinite scroll feeds with cursor-based pagination, create notification systems with delivery guarantees, and deploy content moderation pipelines using AI services. Learn to handle the unique challenges of high-write workloads and eventually consistent social graphs.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='46. e-commerce_store.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='48. social_media_platform.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
